In [331]:
# avoid recursion limit errors
import sys
sys.setrecursionlimit(10000)

In [332]:
import random
import heapq
import math
import sys
import time
from collections import defaultdict, deque, Counter
from itertools import combinations
from IPython.display import clear_output
import timeit

#import numpy as np
#import pandas as pd

# Data Structure

In [333]:
# The node object represent a generic node of the search tree
# each node has a parent , a state, an action and a path_cost

class Node:
    
    #constructor
    def __init__ (self, state, parent = None, action = None, path_cost = 0):
        self.__dict__.update (state = state, parent = parent, action = action, path_cost = path_cost)
        
    #representation
    def __repr__(self):
        return '<{}>'.format(self.state)
    #lenght
    def __len__(self):
        return 0 if self.parent is None else (1+ len(self.parent))
    #less than
    def __lt__(self,node):
        return self.path_cost < node.path_cost
 

In [334]:
#
# Problem 
#
class Problem(object):
    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):  
        raise NotImplementedError
    def result(self, state, action):
        raise NotImplementedError
    def is_goal(self, state):        
        return state == self.goal
    def action_cost(self, s, a, s1):
        return 1
    def h(self, node):              
        return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)

In [335]:
#fail is used to stop the search because the search alg cant find the solution
fail = Node('fail', path_cost=math.inf)
# cutoff node is used
cutoff  = Node('cutoff',  path_cost=math.inf) 

In [336]:
#generate child nodes  
def expandChilds(problem, node):
    s = node.state
    for action in problem.actions(s):
        #from a state s and an action a the problem go to state s1
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)

#sequence actions to reach a node        
def path_actions(node):
    if node.parent is None:
        return []  
    return path_actions(node.parent) + [node.action]

#sequence of states 
def path_states(node):
    if node in (cutoff, fail, None): 
        return []
    return path_states(node.parent) + [node.state]   


In [337]:
#
# Queues
#
FIFOQueue = deque

LIFOQueue = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)
         
    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]
    
    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)

In [338]:
## Search Alg ##

##BFS##


def best_first_search(problem, f):
    exploredCount = 0
    "Search nodes with minimum f(node) value first."
    node = Node(problem.initial)
    horizon = PriorityQueue([node], key=f)
    explored = {problem.initial: node}
    while horizon:
        node = horizon.pop()
        exploredCount= exploredCount+1 
        if problem.is_goal(node.state):
            print ('--------------------------------------')
            print ('Initial State:'+ str(problem.initial))
            print ('Explored Nodes:'+ str(exploredCount))
            print ('Actions:'+ str(len(path_states(node))))
            print ('Cost:'+ str(node.path_cost))
            print ('Final State:'+ str(node.state))
            return node
        for child in expandChilds(problem, node):
            s = child.state
            if s not in explored or child.path_cost < explored[s].path_cost:
                explored[s] = child 
                horizon.add(child)
    #print ('Fail')            
    return fail


def breadth_first_bfs(problem):
    return best_first_search(problem, f=len)

def astar_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=lambda n: g(n) + h(n))

In [339]:
#
# EightPuzzle class
#
class EightPuzzle(Problem):
    """ The problem of sliding tiles numbered from 1 to 8 on a 3x3 board,
    where one of the squares is a blank, trying to reach a goal configuration.
    A board state is represented as a tuple of length 9, where the element at index i 
    represents the tile number at index i, or 0 if for the empty square, e.g. the goal:
        1 2 3
        4 5 6 ==> (1, 2, 3, 4, 5, 6, 7, 8, 0)
        7 8 _
    """

    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 0)):
        assert inversions(initial) % 2 == inversions(goal) % 2 # Parity check
        self.initial, self.goal = initial, goal
    
    def actions(self, state):
        """The indexes of the squares that the blank can move to."""
        moves = ((1, 3),    (0, 2, 4),    (1, 5),
                 (0, 4, 6), (1, 3, 5, 7), (2, 4, 8),
                 (3, 7),    (4, 6, 8),    (7, 5))
        blank = state.index(0)
        return moves[blank]
    
    def result(self, state, action):
        """Swap the blank with the square numbered `action`."""
        s = list(state)
        blank = state.index(0)
        s[action], s[blank] = s[blank], s[action]
        return tuple(s)
    
    def h1(self, node):
        """The misplaced tiles heuristic."""
        return hamming_distance(node.state, self.goal)
    
    def h2(self, node):
        """The Manhattan heuristic."""
        X = (0, 1, 2, 0, 1, 2, 0, 1, 2)
        Y = (0, 0, 0, 1, 1, 1, 2, 2, 2)
        return sum(abs(X[s] - X[g]) + abs(Y[s] - Y[g])
                   for (s, g) in zip(node.state, self.goal) if s != 0)
    
    def h(self, node): return self.h2(node)
    
    
def hamming_distance(A, B):
    "Number of positions where vectors A and B are different."
    return sum(a != b for a, b in zip(A, B))
    

def inversions(board):
    "The number of times a piece is a smaller number than a following piece."
    return sum((a > b and a != 0 and b != 0) for (a, b) in combinations(board, 2))
    
    
def board8(board, fmt=(3 * '{} {} {}\n')):
    "A string representing an 8-puzzle board"
    return fmt.format(*board).replace('0', '_')

In [340]:
e1 = EightPuzzle((1, 4, 2, 0, 7, 5, 3, 6, 8))
e2 = EightPuzzle((0, 1, 2, 3, 4, 5, 6, 7, 8))
e3 = EightPuzzle((4, 0, 2, 5, 1, 3, 7, 8, 6))
e4 = EightPuzzle((7, 2, 4, 5, 0, 6, 8, 3, 1))
e5 = EightPuzzle((8, 6, 7, 2, 5, 4, 3, 0, 1))

In [341]:
problems = [e1, e2, e3, e4, e5]
#problems = [e1]

In [342]:
breadth_first_bfs(e1)

--------------------------------------
Initial State:(1, 4, 2, 0, 7, 5, 3, 6, 8)
Explored Nodes:105408
Actions:24
Cost:23
Final State:(1, 2, 3, 4, 5, 6, 7, 8, 0)


<(1, 2, 3, 4, 5, 6, 7, 8, 0)>

In [343]:
#
# CountCalls class
#
class CountCalls:
    """Delegate all attribute gets to the object, and count them in ._counts"""
    def __init__(self, obj):
        self._object = obj
        self._counts = Counter()
        
    def __getattr__(self, attr):
        "Delegate to the original object, after incrementing a counter."
        self._counts[attr] += 1
        return getattr(self._object, attr)

        
def report(searchers, problems, verbose=True):
    """Show summary statistics for each searcher (and on each problem unless verbose is false)."""
    for searcher in searchers:
        print(searcher.__name__ + ':')
        total_counts = Counter()
        for p in problems:
            prob = CountCalls(p)
            
            t0 = time.time()
            soln = searcher(prob)
            t1 = time.time() - t0
            
            counts = prob._counts; 
            counts.update(actions=len(soln), cost=soln.path_cost, seconds=t1)
            total_counts += counts
            if verbose: report_counts(counts, str(p)[:40])
        #report_counts(total_counts, 'TOTAL\n')
        
def report_counts(counts, name):
    """Print one line of the counts report."""
    print('{:9,d} nodes |{:9,d} goal |{:5.0f} cost |{:8,d} actions | {:.4f} seconds | {}'.format(
          counts['result'], counts['is_goal'], counts['cost'], counts['actions'], counts['seconds'], name))  

def report (search, problem, verbose = True):
    prob = CountCalls(problem)
    startTime = time.time()
    sol = search (problem)
    totTime = time.time() - startTime
    total_counts = Counter()
    #print ('--------------------------------------')
    #print ('Initial State:'+ str(problem.initial))
    #print ('Actions:'+ str(len(sol)))
    #print ('Cost:'+ str(sol.path_cost))
    #print ('Time:'+ str(totTime))
    #print ('Problem:'+ str(getattr(problem.result)))
    counts = prob._counts; 
    counts.update(actions=len(sol), cost=sol.path_cost, seconds=totTime)
    total_counts += counts
    if verbose: report_counts(counts, str(problem)[:40])
    report_counts(total_counts, 'TOTAL\n')

In [346]:
report ([breadth_first_bfs],problems)

breadth_first_bfs:
--------------------------------------
Initial State:(1, 4, 2, 0, 7, 5, 3, 6, 8)
Explored Nodes:105408
Actions:24
Cost:23
Final State:(1, 2, 3, 4, 5, 6, 7, 8, 0)
  285,089 nodes |  105,408 goal |   23 cost | 105,430 actions | 3.1020 seconds | EightPuzzle((1, 4, 2, 0, 7, 5, 3, 6, 8),
--------------------------------------
Initial State:(0, 1, 2, 3, 4, 5, 6, 7, 8)
Explored Nodes:80042
Actions:23
Cost:22
Final State:(1, 2, 3, 4, 5, 6, 7, 8, 0)
  217,148 nodes |   80,042 goal |   22 cost |  80,063 actions | 2.2285 seconds | EightPuzzle((0, 1, 2, 3, 4, 5, 6, 7, 8),
--------------------------------------
Initial State:(4, 0, 2, 5, 1, 3, 7, 8, 6)
Explored Nodes:139
Actions:8
Cost:7
Final State:(1, 2, 3, 4, 5, 6, 7, 8, 0)
      378 nodes |      139 goal |    7 cost |     145 actions | 0.0040 seconds | EightPuzzle((4, 0, 2, 5, 1, 3, 7, 8, 6),
--------------------------------------
Initial State:(7, 2, 4, 5, 0, 6, 8, 3, 1)
Explored Nodes:55425
Actions:21
Cost:20
Final State:(1